## **Message Roles**

<img src='./images/system-roles.png' width=75% height=75%>

* __System message__: guides model behavior
* __User message__: prompt from the user.
* __Assistant message__: response to user propmt

To ask "What is prompt engineering?" via the API, we set up the client by calling the OpenAI class and setting the api_key. Then, we call the chat.completions.create() function, specifying the model and messages as a list of dictionaries with the role and content. We set the temperature to zero for deterministic answers and do not specify max_tokens to allow full responses. Finally, we print the response.

```python
prompt = "What is prompt engineering?"

client = OpenAI(api_key="<OPENAI_API_KEY>))
response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages[
        {"role": "user",
        "content": prompt}],
        temperature = 0
)

print(response.choices[0].message.content)
```

Output: <br>
<img src="./images/prompt-engineering-response.png" width=90% height=90%>

To avoid writing it every time, we wrap the code inside a function named `get_response` that takes a prompt as input and sends it to the API just as before. Instead of printing the response, the function returns it as output. Now, we can call this function with one line of code and save the returned value in a variable we can print.

```python
def get_response(prompt):
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [
            {'role': 'user',
            'content': prompt}],
            temperature = 0
    )
    return response.choices[0].message.content

prompt = "What is prompt engineering?"
response = get_response(prompt)
print(response)
```

The output is the same as before. <br>
<img src="./images/prompt-engineering-response.png" width=90% height=90%>

## **Key principles of prompt engineering**

### **Using action verbs**

When asking the model to perform certain tasks, opt for action verbs that explicitly guide the model's task. Conversely, avoid using ambiguous verbs.

<table style="width:40%; border-collapse: collapse; margin: auto;">
  <tr>
    <th style="background-color: dodgerblue; color: white; padding: 10px; border: 1px solid black; text-align: center;">Verbs to Use</th>
    <th style="background-color: lightcoral; color: black; padding: 10px; border: 1px solid black; text-align: center;">Verbs to Avoid</th>
  </tr>
  <tr>
    <td style="padding: 10px; border: 1px solid black; text-align: center;">write</td>
    <td style="padding: 10px; border: 1px solid black; text-align: center;">understand</td>
  </tr>
  <tr>
    <td style="padding: 10px; border: 1px solid black; text-align: center;">complete</td>
    <td style="padding: 10px; border: 1px solid black; text-align: center;">think</td>
  </tr>
  <tr>
    <td style="padding: 10px; border: 1px solid black; text-align: center;">explain</td>
    <td style="padding: 10px; border: 1px solid black; text-align: center;">feel</td>
  </tr>
  <tr>
    <td style="padding: 10px; border: 1px solid black; text-align: center;">describe</td>
    <td style="padding: 10px; border: 1px solid black; text-align: center;">try</td>
  </tr>
  <tr>
    <td style="padding: 10px; border: 1px solid black; text-align: center;">evaluate</td>
    <td style="padding: 10px; border: 1px solid black; text-align: center;">know</td>
  </tr>
  <tr>
    <td style="padding: 10px; border: 1px solid black; text-align: center;">...</td>
    <td style="padding: 10px; border: 1px solid black; text-align: center;">...</td>
  </tr>
</table>

A prompt asking the model to `think about deforestation` is ineffective. While the model generates some output, it remains vague and the prompt needs greater clarity on what is being asked. For example, we can use an action verb to guide the model to a specific task: `proposing strategies to reduce deforestation`.

When crafting prompts, we'll want to provide specific, descriptive, and detailed instructions about:
* context
* output length
* format and style
* audience

A prompt asking to tell us about dogs is too broad. However, when we ask:

```python
prompt = "Write a descriptive paragraph about the behavior and characteristics of Golden Retrievers, highlighting their friendly nature, intelligence, and suitability as family pets."
```

we get a more precise answer with the details we've requested, including characteristics and suitability for families.


__Crafting a well-structured prompt with delimiters__

* Start prompt with instructions
* Use delimiters (parentheses, brackets, backticks etc.) to specify input parts*
* Mention which delimiters are used
 
This helps the model locate the input. 
In the following example, we use triple backticks to delimit input text for summarization.

```python
prompt = """Summarize the text delimited by triple backticks into bullet points.
        ```TEXT GOES HERE```"""
response =get_response(prompt)
```

__Using f-strings__

Instead of including long inputs within the prompt string, we can use Python's f-strings to embed a pre-defined string. In this example we define the variable `text`, add an `'f'` before the prompt string, and place the text variable in curly brackets inside the prompt. This incorporates the variable's content into the prompt. 

```python
text = "This is a sample text to summarize"
prompt = f"""Summarize the text delimited by triple backticks into bullet points.
        ```{text}```"""
print(prompt)
```
Output: <br>
<img src='./images/f-string_prompt.png' width=60% height=60%>


### **Structured putputs and conditional prompts**

__Structured outputs__

When a custom output format is needed, one approach is to break down the prompt into parts.

Here we define an input text, such as the opening of a fairytale about a boy named David. We then instruct the model to generate an appropriate title, and specify the output format for the text and title. We combine the instructions, output format, and input text in the final prompt. 



```python
text = "Once upon a time in a quaint little village, there lived a curious young boy named David. David was [...]"
instructions = "You will be provided with a text delimited by triple backticks. Generate a suitable title for it."

output_format = """Use the following format for the output:
        - Text: <text we want to title>
        -title: <the generated title>"""

prompt = instructions + output_format + f"```{text}```"
print(get_response(prompt))
```
This ensures the output meets our requirements, providing a title for David's adventures, with the original text, in the requested format.

Output: <br>
<img src='./images/structured_output.png' width=70% height=70%>

### **Conditional prompts**

* Incorporate logic or conditions
* Conditional prompts follow an if-else style

Suppose we want the model to suggest a title for some provided text, but only if the text is in English. To do that, we explicitly mention this condition in the prompt. If the text is in another language, we tell the model to inform users that it only understands English. Now, for a French text describing my favorite season, the output will be `'I only understand English'`.

```python
text = "Le printemps est ma saison préférée. Quand les premières fleurs commencent à éclore, et que les arbres se parent de feuilles vertes et tendres, je me sens revivre [...]"

prompt = f"""You will be provided with a text delimited by triple backticks.
         If the text is written in English, suggest a suitable title for it.
         Otherwise, write 'I only understand English'.
         ```{text}```"""

pring(get_response(prompt))
```

Output: <br>

`I only understand English`

We can incorporate multiple conditions in our prompt. For instance, we might tell the model to generate titles only for English texts containing the keyword 'technology'. <br>
If the text is written in English, we check if it contains the keyword. If it does, the model suggests a title. Otherwise, it responds with `'keyword not found'`.

```python
text = "In the heart of the forest, sunlight filters through the lush green canopy, creating a tranquil atmosphere [...] "

prompt = f"""You will be provided with a text delimited by triple backticks.
         If the text is written in English, check if it contains the keyword 'technology'.
         If it does, suggest a suitable title for it, otherwise, write 'Keyword not found'.
         If the text is not written in English, reply with 'I only understand English'.
         ```{text}```"""
print(get_response(prompt))
```

Output: <br>
`Keyword not found`

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load the environment variables
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)
text = "The sun was setting behind the mountains, casting a warm golden glow across the landscape."

def get_response(prompt):
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [
            {'role': 'user',
            'content': prompt}],
            temperature = 0
    )
    return response.choices[0].message.content

# Create the instructions
instructions = """You will be provided with a text delimited by three backtips.
               Infer the language, and the number of the sentences of the text.
               If the number of sentences is more than one, generate a suitable title for it.
               Otherwise, write 'N/A'.
               """

# Create the output format
output_format = """Use the following format for the output:
                - Text: <the text you are provided>
                - Nr. of sentences: <number of sentences in the text>
                - Title: <The generated title>
                - Language: <the language of the text>"""

prompt = instructions + output_format + f"```{text}```"
response = get_response(prompt)
print(response)

- Text: The sun was setting behind the mountains, casting a warm golden glow across the landscape.
- Nr. of sentences: 1
- Title: N/A
- Language: English
